In [137]:
# import everything that I will use
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from datetime import date
import datetime
import requests




In [138]:
#open Selenium Chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/luizfernandotoledo/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/ipykernel_18055/1903745058.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [139]:
#find a link that gives me the last 30 days FOIA requests with the keyword 'Bolsonaro'
# if you want to change the keyword, just change the variable keywords
# if you want to change the time range, change the number in parenthesis in the 'days' variable

# creating a link
keyword = 'bolsonaro'
today = date.today()
days = datetime.timedelta(90)
thirty_days_ago = today - days
thirty_days_ago = str(thirty_days_ago)
year_regex = r"(\d\d\d\d)-"
year_only = re.findall(year_regex, thirty_days_ago)[0]
month_regex = r".....(..)"
month_only = re.findall(month_regex, thirty_days_ago)[0]
day_regex = r"........(..)"
day_only = re.findall(day_regex, thirty_days_ago)[0]


### gambiarra monstra pra pegar a URL pq ainda to aprendendo, rs
useful_link1 = 'http://www.consultaesic.cgu.gov.br/busca/SitePages/resultadopesquisa.aspx?k=ALL('
useful_link2 = keyword
useful_link3 = ')%20DataPedido%3E%3D'
useful_link4 = day_only
useful_link5 = '%2F'
useful_link6 = month_only
useful_link7 = '%2F'
useful_link8 = year_only

useful_link = useful_link1 + useful_link2 + useful_link3 + useful_link4 + useful_link5 + useful_link6 + useful_link7 + useful_link8  


In [140]:
#driver opens the link needed in order to find all FOIA requests

driver.get(useful_link)

In [141]:
#step one: create a list with links from all the pages


list_of_links = []
#try to open up to 100 pages in order to find those links
for i in range(100):
    for pedido in driver.find_elements(By.CLASS_NAME, 'ms-srch-ellipsis'):
        links = pedido.find_elements(By.TAG_NAME, 'a')
        for link in links:
            caso = link.get_attribute('href')
            list_of_links.append(caso)
    #click on the next page
    try:
        driver.find_element(By.XPATH, '//*[@id="PageLinkNext"]').click()
        time.sleep(2)
    except:
        break


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=96.0.4664.55)
Stacktrace:
0   chromedriver                        0x0000000102912269 __gxx_personality_v0 + 582729
1   chromedriver                        0x000000010289dc33 __gxx_personality_v0 + 106003
2   chromedriver                        0x000000010245ae28 chromedriver + 171560
3   chromedriver                        0x000000010243c10e chromedriver + 45326
4   chromedriver                        0x00000001024b11dc chromedriver + 524764
5   chromedriver                        0x00000001024bfabf chromedriver + 584383
6   chromedriver                        0x00000001024adc23 chromedriver + 511011
7   chromedriver                        0x000000010248375e chromedriver + 337758
8   chromedriver                        0x0000000102484a95 chromedriver + 342677
9   chromedriver                        0x00000001028ce8ab __gxx_personality_v0 + 305803
1

In [ ]:
#check if we've got everything
list_of_links

In [ ]:
# create a list and fill it with request title, request link and request text

lista_de_pedidos = []
for link in list_of_links:
    lista_do_pedido = []
    driver.get(link)
    resumao = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[10]/div[1]/div[2]/div/div[2]/div[2]/div[1]/table/tbody/tr/td/div/div[3]/div[1]/div/div[1]/span/div[2]/div/div[2]/div/div/div/div[2]/div[1]/div/div/fieldset/div/div[1]/span/textarea')
    lista_do_pedido.append(resumao.text)
    link_do_pedido = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[10]/div[1]/div[2]/div/div[2]/div[2]/div[1]/table/tbody/tr/td/div/div[3]/div[1]/div/div[1]/span/div[2]/div/div[2]/div/div/div/div[2]/div[2]/div/div/fieldset/div/div[3]/table/tbody/tr/td[2]/div/span/textarea')
    lista_do_pedido.append(link_do_pedido.text)
    texto_do_pedido = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[10]/div[1]/div[2]/div/div[2]/div[2]/div[1]/table/tbody/tr/td/div/div[3]/div[1]/div/div[1]/span/div[2]/div/div[2]/div/div/div/div[2]/div[4]/div/div/fieldset/div/div/table/tbody/tr/td[2]/div[1]/span/div')
    lista_do_pedido.append(texto_do_pedido.text)
    lista_de_pedidos.append(lista_do_pedido)

lista_de_pedidos


In [ ]:
import pandas

df = pd.DataFrame(lista_de_pedidos, columns = ['resumo', 'link', 'texto_do_pedido'])

df.to_csv('lista_de_pedidos.csv')

